In [1]:
%cd gspan_mining
from config import parser
from main import main

/data/experiments/DocRED/gSpan/gspan_mining


In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
# This is just really handy if you make changes to the files.
%load_ext autoreload
%autoreload 2

In [13]:
args_str = '-s 3 -e 2 -v True -d True -p True -n 10 -w True /data/experiments/DocRED/DataViewer/results/docred_train.data'
FLAGS, _ = parser.parse_known_args(args=args_str.split())

In [ ]:
gs = main(FLAGS)

## plot graphs in database

In [ ]:
for g in gs.graphs.values():
    g.plot()

In [4]:
from gspan import gSpan
min_support = 30
alt = False

gs = gSpan(
    database_file_name='/data/experiments/DocRED/DataViewer/results/docred_train.data',
    min_support=min_support,
    min_num_vertices=2,
    # max_num_vertices=3,
    min_num_edges=1,
#     max_ngraphs=5,
    is_undirected=False,
    verbose=False,
    visualize=False,
    where=True,
    alternative_support=alt
)

"""Run the gSpan algorithm."""
# gathered = gs.run(gather=True)
# print(len(gathered))
# print(list(pdfs.edge for pdfs in gathered[0]))
gs.run()
print(gs._report_df)
gs._report_df.to_pickle(f'all_patterns_support_{min_support}{"_alt" if alt else ""}.pkl')



     support                                        description  num_vert
1        160              t # 0\nv 0 ORG \nv 1 LOC \ne 0 1 P159         2
2         31  t # 1\nv 0 ORG \nv 1 LOC \nv 2 TIME \ne 0 1 P1...         3
3        939               t # 2\nv 0 ORG \nv 1 LOC \ne 0 1 P17         2
4         67  t # 3\nv 0 ORG \nv 1 LOC \nv 2 PER \ne 0 1 P17...         3
5        116  t # 4\nv 0 ORG \nv 1 LOC \nv 2 TIME \ne 0 1 P1...         3
..       ...                                                ...       ...
468       42            t # 467\nv 0 ORG \nv 1 ORG \ne 0 1 P749         2
469       41            t # 468\nv 0 ORG \nv 1 ORG \ne 0 1 P355         2
470       31           t # 469\nv 0 LOC \nv 1 LOC \ne 0 1 P1376         2
471       32            t # 470\nv 0 ORG \nv 1 ORG \ne 0 1 P127         2
472       35           t # 471\nv 0 MISC \nv 1 MISC \ne 0 1 P31         2

[472 rows x 3 columns]


In [5]:
from perspective import PerspectiveWidget, Table
PerspectiveWidget(gs._report_df)

PerspectiveWidget(columns=['index', 'support', 'description', 'num_vert'])

In [13]:
print(gs._report_df['description'][58])

t # 57
v 0 LOC 
v 1 LOC 
v 2 LOC 
v 3 LOC 
e 0 1 P17
e 1 2 P150
e 2 3 P150


In [6]:
import itertools

with open('/data/experiments/DocRED/DataViewer/results/docred_train.data', 'r', encoding='utf8') as graph:
    transactions = []
    for line in graph:
        line = line.strip()
        t, *rest = line.split()
        if t == 't':
            c = int(rest[-1])
#             if c >= 5:
#                 break
            transactions.append({'v':[], 'e':[]})
        elif t == 'v':
            transactions[c][t].append(rest[-1])
        else: # t == 'e':
            transactions[c][t].append([int(rest[0]), int(rest[1]), rest[2]])
    


# for t in transactions:
#     print()
#     print(t)
#     for i, e in enumerate(t['e']):
#         print(f'#{i}: {t["v"][e[0]]}_{e[0]} {t["v"][e[1]]}_{e[1]} {e[2]}')

#     print()


def get_edge_patterns(pattern):
    pattern_dict = {'v':[], 'e':[]}
    for line in pattern.split('\n'):
        line = line.strip()
        t, *rest = line.split()
        if t == 't':
            continue
        elif t == 'v':
            pattern_dict[t].append(rest[-1])
        else: # t == 'e':
            pattern_dict[t].append([int(rest[0]), int(rest[1]), rest[2]])
    return [(pattern_dict['v'][start], pattern_dict['v'][end], label) for start, end, label in pattern_dict['e']], pattern_dict


def get_candidates(graph, edge_pattern):
    vs = graph['v']
    es = graph['e']
    candidate_edges = []
    for i, (start, end, label) in enumerate(es):
        if (vs[start], vs[end], label) == edge_pattern:
            candidate_edges.append(i)
    return candidate_edges


def trace(graph2, pattern2, steps):
#     vs2 = pattern2['v']
    es2 = pattern2
    candidate_paths = itertools.product(*steps)
    candidate_paths = [p for p in candidate_paths if len(p) == len(set(p))]
    gvs = graph2['v']
    ges = graph2['e']
    out = []
#     print("Candidates:", candidate_paths)
    for cp2 in candidate_paths:
        # Check if it's valid. We trust the labels and the types.
        # This is basically a type of variable resolution, heh.
        variables = dict()
        okay = True
#         print(cp2)
#         print(es2)
        if len(cp2) == len(es2):
            for step, e in zip(cp2, es2):
#                 print(step, e)
                v0, v1, _ = e
                start, end, _ = ges[step]
                if v0 in variables:
#                     print(f"{variables[v0]}!={start} ? (start): {variables}")
                    if variables[v0] != start:
                        okay = False
                        break
                if v1 in variables:
#                     print(f"{variables[v1]}!={end} ? (end): {variables}")
                    if variables[v1] != end:
                        okay = False
                        break

                variables[v0] = start
                variables[v1] = end
            if not okay:
                continue
            if len(set(variables.values())) == len(variables):
                out.append(cp2)
            
    return out


def debug_one(index, verbose=True):
    tot = 0
    pattern = gs._report_df['description'][index]
    support = gs._report_df['support'][index]
    edge_patterns, pattern_dict = get_edge_patterns(pattern)
    if verbose:
        print(pattern)
    for tr in transactions:
        possible_steps = [get_candidates(tr, ep) for ep in edge_patterns]
#         possible_steps = [[10], [3], [5], [9]]
        founds = trace(tr, pattern_dict['e'], possible_steps)
        if verbose:
            print(founds)
        tot += len(founds)
    if verbose:
        print(index, tot, support)


def debug_all():
    for i, (pattern, support) in enumerate(zip(gs._report_df['description'], gs._report_df['support'])):
        tot = 0
        edge_patterns, pattern_dict = get_edge_patterns(pattern)
        for tr in transactions:
            possible_steps = [get_candidates(tr, ep) for ep in edge_patterns]
            founds = trace(tr, pattern_dict['e'], possible_steps)
    #         print(founds)
            tot += len(founds)
        if tot != support:
            print(i, tot, support)
    
# debug_one(19)
# debug_all()

from multiprocessing.dummy import Pool, Lock
def make_corrections(df_in):
    counts = [0 for _ in range(len(df_in['support']))]
    lock = Lock()
    def count_(inp):
#         print(inp)
        i, pattern = inp
        tot = 0
        edge_patterns, pattern_dict = get_edge_patterns(pattern)
        for tr in transactions:
            possible_steps = [get_candidates(tr, ep) for ep in edge_patterns]
            founds = trace(tr, pattern_dict['e'], possible_steps)
    #         print(founds)
            tot += len(founds)
        lock.acquire()
        print(i, tot)
        counts[i] = tot
        lock.release()
    pool = Pool(23)
#     print(list(zip(*enumerate(df_in['description']))))
    pool.map(count_, enumerate(df_in['description']))
    df_in["counts"] = counts
    return df_in
            

    

df2 = make_corrections(gs._report_df)
PerspectiveWidget(df2)



6 95
0 230
1 45
2 1986
120 470
132 55
3 229
18 1021
7 50
84 1870
4 143
72 2282
8 3704
114 692
126 2624
5 147
9 1572
138 167
36 2158
139 147
133 699
10 447
60 572
85 713
134 70
121 106
37 54
73 1115
102 395
54 476
135 109
136 70
137 167
144 123
145 1944
127 320
140 632
128 83
115 468
38 1041
96 988
24 328
146 1206
129 73
130 86
122 108
123 107
25 441
124 6224
125 5873
11 200
147 407
19 653
141 392
97 948
142 76
143 162
156 864
86 948
131 392
108 114
168 247
20 1508
48 177
169 511
49 42
87 1000
170 407
162 1084
26 94
61 204
55 180
171 156
98 305
163 119
164 643
50 464
21 320
157 824
88 883
172 62
165 313
116 174
148 400
51 79
166 401
167 511
109 115
27 95
110 248
39 997
74 1053
52 139
174 290
53 2006
28 415
180 69
111 105
181 62
29 396
182 322
183 1372
184 2016
186 332
187 129
188 54
189 68
190 257
185 3672
191 65
198 317
192 129
199 122
112 106
113 56
193 1039
200 310
204 710
194 650
201 110
205 435
202 42
195 210
203 2578
196 506
197 445
216 49
217 343
218 126
206 160
219 143
210 58
22

PerspectiveWidget(columns=['index', 'support', 'description', 'num_vert', 'counts'])

In [7]:
df2.to_pickle(f'all_patterns_support_{min_support}_wcounts.pkl')